In [40]:
import numpy as np
import pyqubo

In [41]:
def convert_dict(term):
    label = set()
    quadratic = {}
    const = 0.0

    for term_val in term.values():
        # for const
        if isinstance(term_val, (int, float)):
            const = float(term_val)

        # for h or J
        else:
            for key, val in term_val.items():
                # for J
                if isinstance(key, tuple):
                    quadratic[key] = val
                    label |= set(key)

                # for h
                else:
                    quadratic[(key, key)] = val
                    label.add(key)

#     print(label)
#     label = map(str, label)
#     label = {num: lab for lab, num in enumerate(sorted(list(label)))}
    print(label)
    label = list(label)
    print(quadratic)

    spins = len(label)
    mat = np.zeros((spins, spins))
    for k, v in quadratic.items():
        x = label[k[0]]
        y = label[k[1]]
        mat[x, y] = v
        mat[y, x] = v

    return np.triu(mat), const

In [31]:
h = {'h': {'a': 10.0, 'b': 0.0, 30: 1.2}, 'J': {('a', 'b'): 1.0, (4, 30): -2.3}, 'c': 100}

In [32]:
convert_dict(h)

{4, 'a', 30, 'b'}
{('a', 'a'): 10.0, ('b', 'b'): 0.0, (30, 30): 1.2, ('a', 'b'): 1.0, (4, 30): -2.3}


TypeError: list indices must be integers or slices, not str

In [58]:
def convert_pyqubo(term):
    model = term.compile()
    label = model.label2index
    qubo = model.compiled_qubo.qubo
    const = model.compiled_qubo.offset

    print(label)
    print(qubo)
    print(const)
    spins = len(label)
    mat = np.zeros((spins, spins))
    for k_set, v in qubo.items():
        x = k_set[0]
        y = k_set[1]
        mat[label[x], label[y]] = v
        mat[label[y], label[x]] = v

#     if self._model_type == 'ising':
#         w = np.triu(mat, k=1) / 4
#         b_tmp = np.diag(mat)
#         b = (b_tmp + np.sum(mat, axis=1)) / 4
#         c = const + np.sum(b_tmp) / 2 + np.sum(mat) / 4
#         return np.triu(w + np.diag(b)), c

#     else:
#         return np.triu(mat), const
    return mat, const

In [62]:
city = 4
x = pyqubo.Array.create('x', (city, city), 'SPIN')

In [63]:
time_const = pyqubo.Sum(0, city, lambda i: (pyqubo.Sum(0, city, lambda j: x[i, j]) - 1) ** 2)

In [64]:
convert_pyqubo(time_const)

{'x[0][0]': 0, 'x[0][1]': 1, 'x[0][2]': 2, 'x[0][3]': 3, 'x[1][0]': 4, 'x[1][1]': 5, 'x[1][2]': 6, 'x[1][3]': 7, 'x[2][0]': 8, 'x[2][1]': 9, 'x[2][2]': 10, 'x[2][3]': 11, 'x[3][0]': 12, 'x[3][1]': 13, 'x[3][2]': 14, 'x[3][3]': 15}
{('x[0][0]', 'x[0][1]'): 8.0, ('x[0][0]', 'x[0][2]'): 8.0, ('x[0][0]', 'x[0][3]'): 8.0, ('x[0][1]', 'x[0][2]'): 8.0, ('x[0][1]', 'x[0][3]'): 8.0, ('x[0][2]', 'x[0][3]'): 8.0, ('x[1][0]', 'x[1][1]'): 8.0, ('x[1][0]', 'x[1][2]'): 8.0, ('x[1][0]', 'x[1][3]'): 8.0, ('x[1][1]', 'x[1][2]'): 8.0, ('x[1][1]', 'x[1][3]'): 8.0, ('x[1][2]', 'x[1][3]'): 8.0, ('x[2][0]', 'x[2][1]'): 8.0, ('x[2][0]', 'x[2][2]'): 8.0, ('x[2][0]', 'x[2][3]'): 8.0, ('x[2][1]', 'x[2][2]'): 8.0, ('x[2][1]', 'x[2][3]'): 8.0, ('x[2][2]', 'x[2][3]'): 8.0, ('x[3][0]', 'x[3][1]'): 8.0, ('x[3][0]', 'x[3][2]'): 8.0, ('x[3][0]', 'x[3][3]'): 8.0, ('x[3][1]', 'x[3][2]'): 8.0, ('x[3][1]', 'x[3][3]'): 8.0, ('x[3][2]', 'x[3][3]'): 8.0, ('x[0][0]', 'x[0][0]'): -16.0, ('x[0][1]', 'x[0][1]'): -16.0, ('x[0][2]'

(array([[-16.,   8.,   8.,   8.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  8., -16.,   8.,   8.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  8.,   8., -16.,   8.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  8.,   8.,   8., -16.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0., -16.,   8.,   8.,   8.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   8., -16.,   8.,   8.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   8.,   8., -16.,   8.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   8.,   8.,   8., -16.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., -16.,   8.,   8.,
     